# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8ce4853bb0>
├── 'a' --> tensor([[ 0.2376, -1.2458,  0.9708],
│                   [-1.9023,  1.3073,  0.4801]])
└── 'x' --> <FastTreeValue 0x7f8ce4853c70>
    └── 'c' --> tensor([[ 0.0818, -1.9045, -0.7916, -0.7116],
                        [ 0.2512,  0.7212,  0.4555, -1.7708],
                        [ 0.0225,  0.4319,  0.6053, -0.1198]])

In [4]:
t.a

tensor([[ 0.2376, -1.2458,  0.9708],
        [-1.9023,  1.3073,  0.4801]])

In [5]:
%timeit t.a

45.7 ns ± 0.719 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8ce4853bb0>
├── 'a' --> tensor([[-0.4850, -2.7078, -1.0981],
│                   [ 0.8238,  0.1580,  1.0124]])
└── 'x' --> <FastTreeValue 0x7f8ce4853c70>
    └── 'c' --> tensor([[ 0.0818, -1.9045, -0.7916, -0.7116],
                        [ 0.2512,  0.7212,  0.4555, -1.7708],
                        [ 0.0225,  0.4319,  0.6053, -0.1198]])

In [7]:
%timeit t.a = new_value

48.8 ns ± 0.0898 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2376, -1.2458,  0.9708],
               [-1.9023,  1.3073,  0.4801]]),
    x: Batch(
           c: tensor([[ 0.0818, -1.9045, -0.7916, -0.7116],
                      [ 0.2512,  0.7212,  0.4555, -1.7708],
                      [ 0.0225,  0.4319,  0.6053, -0.1198]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2376, -1.2458,  0.9708],
        [-1.9023,  1.3073,  0.4801]])

In [11]:
%timeit b.a

44.3 ns ± 0.242 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[0.8003, 0.0252, 0.3759],
               [1.2069, 1.3344, 0.6981]]),
    x: Batch(
           c: tensor([[ 0.0818, -1.9045, -0.7916, -0.7116],
                      [ 0.2512,  0.7212,  0.4555, -1.7708],
                      [ 0.0225,  0.4319,  0.6053, -0.1198]]),
       ),
)

In [13]:
%timeit b.a = new_value

372 ns ± 1.66 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

626 ns ± 3.39 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

8.5 µs ± 31 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

129 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

126 µs ± 915 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8bf992b940>
├── 'a' --> tensor([[[ 0.2376, -1.2458,  0.9708],
│                    [-1.9023,  1.3073,  0.4801]],
│           
│                   [[ 0.2376, -1.2458,  0.9708],
│                    [-1.9023,  1.3073,  0.4801]],
│           
│                   [[ 0.2376, -1.2458,  0.9708],
│                    [-1.9023,  1.3073,  0.4801]],
│           
│                   [[ 0.2376, -1.2458,  0.9708],
│                    [-1.9023,  1.3073,  0.4801]],
│           
│                   [[ 0.2376, -1.2458,  0.9708],
│                    [-1.9023,  1.3073,  0.4801]],
│           
│                   [[ 0.2376, -1.2458,  0.9708],
│                    [-1.9023,  1.3073,  0.4801]],
│           
│                   [[ 0.2376, -1.2458,  0.9708],
│                    [-1.9023,  1.3073,  0.4801]],
│           
│                   [[ 0.2376, -1.2458,  0.9708],
│                    [-1.9023,  1.3073,  0.4801]]])
└── 'x' --> <FastTreeValue 0x7f8ce4875ca0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

23.4 µs ± 260 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8c45b73fa0>
├── 'a' --> tensor([[ 0.2376, -1.2458,  0.9708],
│                   [-1.9023,  1.3073,  0.4801],
│                   [ 0.2376, -1.2458,  0.9708],
│                   [-1.9023,  1.3073,  0.4801],
│                   [ 0.2376, -1.2458,  0.9708],
│                   [-1.9023,  1.3073,  0.4801],
│                   [ 0.2376, -1.2458,  0.9708],
│                   [-1.9023,  1.3073,  0.4801],
│                   [ 0.2376, -1.2458,  0.9708],
│                   [-1.9023,  1.3073,  0.4801],
│                   [ 0.2376, -1.2458,  0.9708],
│                   [-1.9023,  1.3073,  0.4801],
│                   [ 0.2376, -1.2458,  0.9708],
│                   [-1.9023,  1.3073,  0.4801],
│                   [ 0.2376, -1.2458,  0.9708],
│                   [-1.9023,  1.3073,  0.4801]])
└── 'x' --> <FastTreeValue 0x7f8bf88b0a00>
    └── 'c' --> tensor([[ 0.0818, -1.9045, -0.7916, -0.7116],
                        [ 0.2512,  0.7212,  0.4555, -1.7708],
                 

In [23]:
%timeit t_cat(trees)

21.8 µs ± 261 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

49.5 µs ± 397 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0818, -1.9045, -0.7916, -0.7116],
                       [ 0.2512,  0.7212,  0.4555, -1.7708],
                       [ 0.0225,  0.4319,  0.6053, -0.1198]],
              
                      [[ 0.0818, -1.9045, -0.7916, -0.7116],
                       [ 0.2512,  0.7212,  0.4555, -1.7708],
                       [ 0.0225,  0.4319,  0.6053, -0.1198]],
              
                      [[ 0.0818, -1.9045, -0.7916, -0.7116],
                       [ 0.2512,  0.7212,  0.4555, -1.7708],
                       [ 0.0225,  0.4319,  0.6053, -0.1198]],
              
                      [[ 0.0818, -1.9045, -0.7916, -0.7116],
                       [ 0.2512,  0.7212,  0.4555, -1.7708],
                       [ 0.0225,  0.4319,  0.6053, -0.1198]],
              
                      [[ 0.0818, -1.9045, -0.7916, -0.7116],
                       [ 0.2512,  0.7212,  0.4555, -1.7708],
                       [ 0.0225,  0.4319,  0.6053, -0.1198]],

In [26]:
%timeit Batch.stack(batches)

61.6 µs ± 140 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0818, -1.9045, -0.7916, -0.7116],
                      [ 0.2512,  0.7212,  0.4555, -1.7708],
                      [ 0.0225,  0.4319,  0.6053, -0.1198],
                      [ 0.0818, -1.9045, -0.7916, -0.7116],
                      [ 0.2512,  0.7212,  0.4555, -1.7708],
                      [ 0.0225,  0.4319,  0.6053, -0.1198],
                      [ 0.0818, -1.9045, -0.7916, -0.7116],
                      [ 0.2512,  0.7212,  0.4555, -1.7708],
                      [ 0.0225,  0.4319,  0.6053, -0.1198],
                      [ 0.0818, -1.9045, -0.7916, -0.7116],
                      [ 0.2512,  0.7212,  0.4555, -1.7708],
                      [ 0.0225,  0.4319,  0.6053, -0.1198],
                      [ 0.0818, -1.9045, -0.7916, -0.7116],
                      [ 0.2512,  0.7212,  0.4555, -1.7708],
                      [ 0.0225,  0.4319,  0.6053, -0.1198],
                      [ 0.0818, -1.9045, -0.7916, -0.7116],
                   

In [28]:
%timeit Batch.cat(batches)

116 µs ± 476 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

269 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
